Numpy 버전 issue로 1.23으로 downgrade\
참고: https://github.com/NVIDIA/TensorRT/issues/2567

In [ ]:
# requirement 다운로드
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install numpy==1.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661750 sha256=78e75aae1aeee6ce518993e60276845c23f8276c373e8051f5836b6cdf041ff7
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86

Azure에서 serving하던 모델 다운로드 서비스 문제로 hugging face를 통한 모델 다운로드 버전의 코드 변경\
참고1: https://github.com/SKTBrain/KoBERT/issues/102 \
참고2: https://github.com/SKTBrain/KoBERT/issues/67 \
아래에 있는 KoBERT 관련 코드들도 이에 맞게 수정

In [ ]:
# colab과 google drive 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# KoBERT를 github에서 colab으로 다운로드

# 변경 전)
# !pip install git+https://github.com/SKTBrain/KoBERT.git@master

# 변경 후)
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-0357nruw/kobert-tokenizer_02abb8a68a0f4e95b2e5385d22468b33
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-0357nruw/kobert-tokenizer_02abb8a68a0f4e95b2e5385d22468b33
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=15269d7950d77f25b54b12d09f0de76a1763147b2be2711ff00d09926dbeb75d
  Stored in directory: /tmp/pip-ephem-wheel-cache-8uwodgx7/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
# KoBERT 관련 library import

# 변경 전)
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

# 변경 후)
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
# Transformers 관련 libary import
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
# colab GPU 사용, 실패 시 CPU로 대체
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [ ]:
# 관련 데이터를 save/load할 path
myDrivePath = '/MyDrive/2024-1/AI 프로젝트 입문/colab/'

In [ ]:
# 감성대화 말뭉치 dataset (https://aihub.or.kr/aidata/7978)
train_data = pd.read_excel("/content/drive/" + myDrivePath + "/감성대화말뭉치(최종데이터)_Training.xlsx")
valid_data = pd.read_excel("/content/drive/" + myDrivePath + "/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [ ]:
# NULL value를 ""로 변경
train_data = train_data.fillna("")
valid_data = valid_data.fillna("")

In [ ]:
# 감성대화 말뭉치에서의 각 사람의 문장을 하나로 합침(감정은 동일)
train_data["문장"] = train_data["사람문장1"].astype(str) + train_data["사람문장2"].astype(str) + train_data["사람문장3"].astype(str)
valid_data["문장"] = valid_data["사람문장1"].astype(str) + valid_data["사람문장2"].astype(str) + valid_data["사람문장3"].astype(str)

In [ ]:
# 모든 데이터를 하나로 묶음
concat_data = pd.concat([train_data, valid_data])

In [ ]:
# 데이터 중 발화 문장과 감정이 대분류를 제외한 나머지 열을 버림
concat_data = concat_data[["문장", "감정_대분류"]]

In [ ]:
# column의 이름 변경 및 data cleaning
concat_data = concat_data.rename({"감정_대분류" : "감정"},axis=1)
concat_data["감정"] = concat_data["감정"].apply(lambda x:x.strip())

In [ ]:
# 각 감정별 개수 확인
concat_data["감정"].value_counts()

감정
불안    10433
분노    10417
상처    10150
슬픔    10128
당황     9804
기쁨     7339
Name: count, dtype: int64

In [ ]:
# 정리한 데이터 예제
concat_data.sample(n=10)

,문장,감정
1183,이제 은퇴가 코앞인데 정말 걱정이야.은퇴하기 전에 내가 할 수 있는 다른 일을 알아...,불안
37811,아빠가 많이 아파서 입원해 있어. 정말 우울해.아버지가 좋아하는 전복죽 사서 병원 ...,슬픔
46319,회사에서 점점 치고 올라오는 후배들 때문에 스트레스받아.실력 있는 후배들 때문에 불...,상처
6350,출산 후 와이프가 부쩍 나에게 화내는 일이 많아졌어.어제는 아무것도 안 했는데 갑자...,당황
2790,하루에 열 두 시간씩 일을 하니까 몸이 성한 곳이 없어.남편이 다쳐서 내가 대신 돈...,상처
29972,주식으로 재산을 탕진한 상태야 정말 난 걱정이다.아직 몸은 건강하니까 닥치는대로 일...,분노
1563,친한 친구들도 분명히 내가 없는 곳에서는 나를 욕하고 있을 거야. 슬프지만 사실이야...,슬픔
9071,새로 들어간 취미 모임에서 웃음이 많고 밝은 사람을 만났는데 요즘 너무 행복해.그 ...,기쁨
2875,아내가 골다공증 때문에 산을 못 가서 매일 나랑 산책하러 가자고 해. 솔직히 너무 ...,분노
47710,엄마가 나랑 가장 친한 친구가 불량하다고 같이 놀지 말래. 속상해.그래서 엄마한테 ...,당황


In [ ]:
# 각 감정을 array로 변경
emotion_array = concat_data["감정"].unique()

In [ ]:
# 감정의 개수 및 종류 확인(이후 class의 개수를 정할 때 사용)
print(emotion_array)

['분노' '기쁨' '불안' '당황' '슬픔' '상처']


In [ ]:
# column이 string type의 특정 감정에 해당하는 경우, array에서의 해당 감정에 대한 index로 변경
for index, emotion in enumerate(emotion_array):
  concat_data.loc[(concat_data["감정"] == emotion), "감정"] = index

In [ ]:
# 감정이 인덱스에 맞게 변경된 것을 확인
concat_data["감정"].value_counts()

감정
2    10433
0    10417
5    10150
4    10128
3     9804
1     7339
Name: count, dtype: int64

In [ ]:
# 각 문장과 감정을 묶어 string으로 구성된 list로 만듦
data_list = []
for q, label in zip(concat_data['문장'], concat_data['감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# data_list의 format 확인(2차원 list로 각각 string type의 문장과 감정 인덱스 저장)
data_list[:5]

[['일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.', '0'],
 ['이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.',
  '0'],
 ['회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. 잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대하는 게 덜 스트레스받을 것 같아.',
  '0'],
 ['직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.',
  '0'],
 ['얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!',
  '0']]

In [ ]:
# train할 데이터와 test로 쓸 데이터 분리
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=1)

In [ ]:
# 분리된 데이터 개수 및 비율 확인
print("train data:", len(dataset_train))
print("test data:", len(dataset_test))
# 위에서 정의한 test_size와 비슷하게 나옴
print("ratio:", len(dataset_test) / (len(dataset_test) + len(dataset_train)))

train data: 46616
test data: 11655
ratio: 0.2000137289560845


In [ ]:
# BERTSentenceTransform를 변경된 KoBERT tokenizer에 맞게 수정
# docs: https://nlp.gluon.ai/api/data.html#gluonnlp.data.BERTSentenceTransform
# 원래 코드: https://nlp.gluon.ai/_modules/gluonnlp/data/transforms.html#BERTSentenceTransform
class KoBERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    vocab : Vocab
        The vocabulary which has cls_token and sep_token registered.
        If vocab.cls_token is not present, vocab.bos_token is used instead.
        If vocab.sep_token is not present, vocab.eos_token is used instead.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab=None, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = self._tokenizer.vocab if vocab is None else vocab
        # RoBERTa does not register CLS token and SEP token
        if hasattr(self._vocab, 'cls_token'):
            self._cls_token = self._vocab.cls_token
        else:
            self._cls_token = self._vocab.bos_token
        if hasattr(self._vocab, 'sep_token'):
            self._sep_token = self._vocab.sep_token
        else:
            self._sep_token = self._vocab.eos_token
        self._padding_token = self._vocab.padding_token

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        If vocab.cls_token and vocab.sep_token are not present,
        vocab.bos_token and vocab.eos_token are used instead.

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        # 수정된 부분(https://github.com/SKTBrain/KoBERT/issues/104#issuecomment-1544185148)
        # 변경 전)
        # tokens_a = self._tokenizer(text_a)
        # 변경 후)
        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        tokens.append(self._cls_token)
        tokens.extend(tokens_a)
        tokens.append(self._sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(self._sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._vocab[tokens]

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([self._vocab[self._padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
# BERT data set
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        # 위에서 KoBERT에 맞게 재정의한 BERTSentenceTransform을 사용
        transform = KoBERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        # 각각의 문장과 그 감정의 인덱스를 feature와 label로서 저장
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
# num_epochs = 15
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERT 모델, Vocabulary 불러오기

# 변경 전)
# tokenizer = get_tokenizer()
# bertmodel, vocab = get_pytorch_kobert_model()

# 변경 후)
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
# tokenize
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
print(data_train[0])

(array([   2, 1986, 6328, 1369, 5770, 1655, 6896, 4656, 6116, 2010, 5782,
       5330, 5760, 4688, 3818, 1407,  862,  832,   54, 6875, 6036, 1952,
       4074, 6745, 6706, 7096, 4593, 7819, 3155, 4656, 6079, 1655, 5017,
       5838, 2856, 7096, 3084, 6060,   54, 5649, 5770, 6142, 4656, 6150,
       2010, 7318, 3149,  777, 5788, 7828,  517, 6539, 5920, 7044, 7483,
       6006, 4461, 6183, 6856, 5405, 6855,   54,    3,    1], dtype=int32), array(63, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 2)


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# KoBERT 학습모델
class BERTClassifier(nn.Module):
    def __init__(self,
                        bert,
                        hidden_size = 768,
                        num_classes=2,    # 감정 개수만큼 클래스 수 조정
                        dr_rate=None,
                        params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        #bert model returns 'last_hidden_state' and 'pooler_output'
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 위에서 정의한 KoBERT 모델 object
model = BERTClassifier(bertmodel,  num_classes=len(emotion_array), dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/drive/" + myDrivePath
ckpt_name=ckpt_path+"saved_model.pt"

In [ ]:
# 모델 학습

# num_epochs = 15
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        # reset gradient
        optimizer.zero_grad()

        # forwarding
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # loss_function: cross entropy
        loss = loss_fn(out, label)

        # back propagation
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)

        if batch_id % log_interval == 0:
            print(" epoch: {}, batch id: {}, loss: {}, train acc: {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch: {} train acc: {}".format(e+1, train_acc / (batch_id+1)))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        test_acc += calc_accuracy(out, label)
    print("epoch: {} test acc: {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-37-7fbd505e1840>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/729 [00:00<?, ?it/s]

 epoch: 1, batch id: 1, loss: 1.8226182460784912, train acc: 0.171875


KeyboardInterrupt: 

In [ ]:
# 모델 저장(save)
torch.save(model.state_dict(), ckpt_name)

In [ ]:
# 모델 불러오기(load)
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [ ]:
# 모델 검증(inference)

def predict(predict_sentence):
    data_set = [[predict_sentence, '0']]

    test_set = BERTDataset(data_set, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

    model.eval()
    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            test_eval=[]
            for i in out:
                logits=i
                logits = logits.detach().cpu().numpy()
                test_eval.append(emotion_array[np.argmax(logits)])

            print(">> 입력하신 내용에서 " + test_eval[0] + "(이)가 느껴집니다.")

In [ ]:
# TEST
sentence = input("하고싶은 말을 입력해주세요 : ")
if sentence != 0 :
  predict(sentence)